In [1]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


#Setup of some modules


In [2]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get("wandb"))
hf_token=userdata.get("huggingFace")

!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: malek-bendhia (malek-bendhia-fsb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llmFineTuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llmFineTuning`


In [4]:
!pip install json_repair

In [5]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests
import torch

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


data_dir="/gdrive/MyDrive/llmFinetuning"
base_model_id="Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype=None


In [6]:
facture="""AZUR C90129 OFFICE SW LA NOUVELLE VISION DE LEDUCATION Tél: Fax: CodeT.V.A1824128EAM000 Facture NUMERO DATE REFERENCE BLMN° FV230257 05/10/23 Référence Désignation Qté Px unitaire Remise Montant HT TVA DEL391013 UNITE DELLVOSTRO391013-12700/4G/1TO 1,00 DELE2422H 1000,000 1000,000 C07 ECRAN DELL24"E2422HN 1,00 415,000 415,000 C07 Code Base Taux Montant Total HT Total Remise Total TTC Timbre Fiscal NETAPAYER C07 1415,000 7,00% 99,050 1415,000 0,000 1515,050 1,000 1515,050 TFV 0,000 1,000 Arrétée la présente facture a la somme de: Mille cinq cent quinze Dinars cinquante Millimes Total 1415,000 100,050 AZUR OFFICE BureautigueetHigh Teci Conditions de réglement: 1515.050 05/10/23 azuroffice2018@gmail.com F:1568667/YB//M/000 Nb-Toute reclamation doit nous parvenir dans les 48 heures -Prix qualité&quantité conforme MF1568667/Y/BM000-RCB07111712018 27268042/21341890 f O RIB:BIATHAMMAMET 0890901003100292061 azuroffice2018@gmail.com
"""

#Data Extraction

###Pydantic is the most famous schema manager in python ..S’assurer que les données sont bien du bon type

In [7]:
class InvoicesDetails(BaseModel):
    supplier_name: str = Field(..., min_length=2, max_length=300,
                               description="Nom de la société fournisseur")

    address: str = Field(..., min_length=5, max_length=500,
                         description="Adresse de la société fournisseur")

    invoice_number: str = Field(..., min_length=1, max_length=100,
                                description="Numéro de la facture")

    invoice_date: str = Field(...,
                               description="Date de la facture")

    client_name: str = Field(..., min_length=2, max_length=300,
                             description="Nom du client")

    total_ht: float = Field(..., gt=0,
                            description="Montant Total Hors Taxes")

    timbre_consumption: Optional[float] = Field(0.0, ge=0,
                                                description="Droit de timbre sur consommation")

    timbre_fiscal: Optional[float] = Field(0.0, ge=0,
                                           description="Timbre fiscal")

    total_ttc: float = Field(..., gt=0,
                             description="Montant TTC (Toutes Taxes Comprises)")

    tva_percent: float = Field(..., ge=0, le=100,
                               description="Taux de TVA en pourcentage")

    tva: float = Field(..., ge=0,
                       description="Montant de la TVA")

    mf_client: Optional[str] = Field(None,
                                     description="Matricule fiscal du client")

    mf_supplier: Optional[str] = Field(None,
                                       description="Matricule fiscal du fournisseur")


In [8]:
details_extraction_messages=[
    {
        "role": "system",
        "content":"\n".join([
            "You are an NLP data parser.",
            "You will be provided by a text associated with a Pydantic scheme.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text. ",
            "Do not generate any introduction or conclusion. "
        ])
    },
    {
        "role":"user",
        "content":"\n".join([
            "## facture :",
            facture.strip(),
            "",
            "## Pydantic details:",
            json.dumps(
                InvoicesDetails.model_json_schema(), ensure_ascii=False
                ),
            "",
            "## Story Details:",
            "```json"
        ])
    }
]

#Evaluation

In [9]:
 model= AutoModelForCausalLM.from_pretrained(
     base_model_id,
     device_map="auto",
     torch_dtype = torch_dtype
 )

 tokenizer=AutoTokenizer.from_pretrained(base_model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
text=tokenizer.apply_chat_template(
    details_extraction_messages,
    add_generation_prompt=True,
    tokenize=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)


generated_ids=model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None , top_p=None
)

generated_ids=[
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response= tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
print(response)

{
    "supplier_name": "AZUR",
    "address": "OFFICE SW LA NOUVELLE VISION DE LEDUCATION",
    "invoice_number": "FV230257",
    "invoice_date": "05/10/23",
    "client_name": "AZUR OFFICE",
    "total_ht": 1415,
    "timbre_consumption": 0.0,
    "timbre_fiscal": 1.0,
    "total_ttc": 1515.05,
    "tva_percent": 100,
    "tva": 151.505,
    "mf_client": "MF1568667/YB//M/000",
    "mf_supplier": "TFV 0,000"
}
